## Análisis Comercial de clientes

In [2]:
# 1. Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
from io import StringIO

# Configuración visual
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="muted", font_scale=1.1)


In [15]:
# Rango de Fechas

# El uso de la central "Anura" arranca el 26-05-2021

fecha_inicio = "2023-01-01 00:01"
fecha_fin = "2024-12-31 23:59"

## Carga de Datos

In [ ]:
# Cargar desde CSV (puedes adaptarlo para leer desde BD o Excel)
# El CSV debería tener columnas como: Cliente, Licencias, Productos, Tickets, Llamadas, Antiguedad_Anios

"""

df = pd.read_csv("clientes.csv")

print("Vista previa de datos:")
display(df.head())
print("\nInformación general:")
print(df.info())

"""



In [16]:
# LLAMADAS desde Anura

def generarDS_Anura(start_date, end_date):
    """
    Genera un request a la API de Anura para obtener un DataSet que luego procesaremos.
    Parámetros: 
        start_date: fecha inicio
        end_date: fecha final
    Retorno:
        df : DataFrame que sería la sábana de datos en el rango de fecha establecido.
    """
    
    # Datos de autenticación y solicitud para obtener el archivo CSV
    CLIENT_ID = "sc_5187_etsol"
    CLIENT_PASSWORD = "f413f20c-e550-46b2-a992-3e6682c66176"

    url_token = "https://sso.anura.com.ar/auth/realms/anura/protocol/openid-connect/token"
    data = {
        "grant_type": "client_credentials",
        "scope": "offline_access"
    }

    response = requests.post(url_token, data=data, auth=(CLIENT_ID, CLIENT_PASSWORD))
    refresh_token = response.json()['refresh_token']

    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": CLIENT_ID
    }

    response = requests.post(url_token, data=data, auth=(CLIENT_ID, CLIENT_PASSWORD))
    access_token = response.json()['access_token']

    url_gcapi = "https://api.anura.com.ar/GCAPI/JSON-RPC"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    payload = {
        "id": 6,
        "method": "AdminService.getDownloadToken",
        "params": []
    }

    response = requests.post(url_gcapi, headers=headers, json=payload)
    token_gcapi = response.json()['result']

    # se define fecha antes de entrar a este segmento de código.
    """ start_date = "2024-05-04 00:00"
    end_date = "2024-05-04 23:59" """

    url_export_cdrs = f"https://api.anura.com.ar/GCAPI/Download/ExportCdrs?Authorization={token_gcapi}&startDate={start_date}&endDate={end_date}&filter=&extraInfo=true&accountId="
    response = requests.get(url_export_cdrs)

    # Leer los datos en un DataFrame
    df = pd.read_csv(StringIO(response.content.decode('unicode_escape'))) # Utilizo la misma codificación que uso en el principal "telefonia.ipynb"

    # Verificamos que haya datos para procesar
    if df.shape[0] == 0:
        print("Dataset vacío .......")
    
    return df # Retorno con el df para depurarlo y comenzar a trabajarlo.

In [17]:
ds_llamadas = generarDS_Anura(fecha_inicio, fecha_fin)
display(ds_llamadas)

,Fecha,DirecciÃ³n,Nro. Origen,Origen,Nro. Destino,Destino,Estado,DuraciÃ³n total,DuraciÃ³n conversaciÃ³n,Tiempo espera,Precio,Ultima acciÃ³n,Cola nombre,Cola estado,Cola duraciÃ³n total,Cola duraciÃ³n conversaciÃ³n,Cola tiempo espera,Cola agente,Cola terminal agente,Cuenta contestÃ³,Terminal contestÃ³,Departamento
0,2024-07-01 07:45:56.378,IN,3415962322,Argentina/ROSARIO - MOVIL,01170780972,Preatendedor,ANSWER,25,25,0,0.0,IVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
1,2024-07-01 07:46:20.704,OUT,3415962322,NaN,800,Soporte2,ANSWER,11,11,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
2,2024-07-01 07:46:20.734,IN,3415962322,Argentina/ROSARIO - MOVIL,800,Soporte2,ANSWER,11,11,0,0.0,PLAYBACK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
3,2024-07-01 07:46:39.181,IN,3415962322,Argentina/ROSARIO - MOVIL,01170780972,Preatendedor,ANSWER,12,12,0,0.0,IVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
4,2024-07-01 07:46:51.222,OUT,3415962322,NaN,800,Soporte2,ANSWER,11,11,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36483,2024-12-31 13:22:20.498,IN,1132022600,Argentina/AMBA - MOVIL,800,Soporte2,ANSWER,30,30,0,0.0,IVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
36484,2024-12-31 13:22:49.92,OUT,1132022600,NaN,800,Soporte2,ANSWER,52,52,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
36485,2024-12-31 13:22:49.953,IN,1132022600,Argentina/AMBA - MOVIL,800,Soporte2,ANSWER,40,40,0,0.0,IVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte
36486,2024-12-31 13:23:29.902,OUT,1132022600,NaN,800,Soporte2,ANSWER,12,12,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mesa de Soporte


## 3. Limpieza y preparación de los datos


In [ ]:
# Eliminar duplicados
df = df.drop_duplicates()

# Manejo de valores nulos
df = df.fillna(0)

# Crear métricas derivadas
df["Solicitudes_Total"] = df["Tickets"] + df["Llamadas"]
df["Solicitudes_por_Licencia"] = df["Solicitudes_Total"] / df["Licencias"].replace(0, np.nan)
df["Productos_por_Año"] = df["Productos"] / df["Antiguedad_Anios"].replace(0, np.nan)



In [ ]:
# =====================================================
# 4. ANÁLISIS EXPLORATORIO
# =====================================================

# Estadísticas descriptivas
print("\nEstadísticas generales:")
display(df.describe())

# Distribución de clientes por licencias
plt.figure(figsize=(8,5))
sns.histplot(df["Licencias"], bins=20, kde=True)
plt.title("Distribución de Licencias por Cliente")
plt.show()

# Relación Licencias vs. Tickets
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x="Licencias", y="Tickets")
plt.title("Licencias vs Tickets")
plt.show()

# Pareto: clientes que más solicitudes generan
df_pareto = df.sort_values("Solicitudes_Total", ascending=False)
df_pareto["% Acumulado"] = df_pareto["Solicitudes_Total"].cumsum() / df_pareto["Solicitudes_Total"].sum() * 100

fig = px.bar(df_pareto, x="Cliente", y="Solicitudes_Total", title="Pareto de Solicitudes por Cliente")
fig.add_scatter(x=df_pareto["Cliente"], y=df_pareto["% Acumulado"], mode="lines+markers", name="% Acumulado")
fig.show()



In [ ]:
# =====================================================
# 5. SEGMENTACIÓN DE CLIENTES (CLUSTERING)
# =====================================================

# Seleccionar variables para el modelo
vars_modelo = ["Licencias", "Productos", "Solicitudes_Total", "Antiguedad_Anios"]
X = df[vars_modelo]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Visualización de clusters
fig = px.scatter_3d(df, x="Licencias", y="Productos", z="Solicitudes_Total",
                    color="Cluster", hover_data=["Cliente"])
fig.show()



In [ ]:
# =====================================================
# 6. INSIGHTS INICIALES
# =====================================================
# Agrupar por cluster para ver promedios
insights = df.groupby("Cluster")[vars_modelo].mean()
print("\nPromedios por cluster:")
display(insights)

# Guardar resultados
df.to_csv("clientes_segmentados.csv", index=False)
print("\nArchivo 'clientes_segmentados.csv' generado.")